In [5]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta
import json

# Initialize a BlobServiceClient
connect_str = "DefaultEndpointsProtocol=https;AccountName=aimemes;AccountKey=sgmw4f5fJMc0kIXueL0nONs6z1k8bMV/V82osrVLtwtdJl44AVt7Sjr42UcHSBwxuU93sH65cZqj+AStdhTOLg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of your container
container_name = 'memesimages'

# Name of the folder you want to list items from
folder_name = 'meme_images/'

# Initialize a ContainerClient
container_client = blob_service_client.get_container_client(container_name)

# Generate a SAS token that expires in 10 years
def generate_sas_for_blob(blob_name):
    sas_token = generate_blob_sas(account_name=blob_service_client.account_name,
                                   container_name=container_name,
                                   blob_name=blob_name,
                                   account_key=connect_str.split("AccountKey=")[1].split(";")[0],
                                   permission=BlobSasPermissions(read=True),
                                   expiry=datetime.utcnow() + timedelta(days=365 * 10))  # 10 years
    return sas_token

# List all blobs in the folder and generate data for JSON
json_data = []
for blob in container_client.list_blobs(name_starts_with=folder_name):
    sas_token = generate_sas_for_blob(blob.name)
    blob_url_with_sas = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob.name}?{sas_token}"
    data = {
        "data": {
            "url": blob_url_with_sas,
            "q1": "Describe why this meme is funny",
            "q2": "Describe the metaphors in this meme",
            "q3": "Give examples of a caption of this meme"
        }
    }
    json_data.append(data)

# Write the data to a JSON file
with open('meme_images.json', 'w') as f:
    json.dump(json_data, f, indent=4)
